In [ ]:
import folium
import data.DownloadData as DD
import data.DataSorting as DS
import json
import geojson
from urllib.request import urlopen
from geojson_utils import centroid

In [ ]:
# récupération des contours cantonaux
with urlopen('https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/cantons.geojson') as f:
    cantons = geojson.load(f)

In [ ]:
# récupération des données sur les stations-service
DD.get_ten_Mins_Data()
DD.get_dailyData()

# on récupère le J-5 pour observer la variation 
chaine = DD.get_chaine()

# calcul des prix moyen pour les cantons
can_prices = DS.DictCantons("PrixCarburants_instantane.xml")
can_prices_B = DS.DictCantons("PrixCarburants_quotidien_" + chaine + ".xml")

# calcul des variations
variations_can = {key: can_prices[key] - can_prices_B.get(key, 0) for key in can_prices.keys()}

In [ ]:
# tendance cantonale
average_can = folium.Map(location=[46.232192999999995, 2.209666999999996], zoom_start=5)

folium.Choropleth(
    geo_data=cantons,
    name="choropleth",
    data=can_prices,
    key_on="feature.properties.nom",
    fill_color="YlOrRd",
    nan_fill_color="White", #Use white color if there is no data available for the departements
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Diesel prices in France cantons", #title of the legend
    highlight=True,
    line_color='black'
).add_to(average_can)

folium.LayerControl().add_to(average_can)

average_can

In [ ]:
# ajout des flèches de variation
for i in range (0, len(cantons["features"])):
    if cantons["features"][i].geometry.type == 'Polygon':
        box_str = str(cantons["features"][i].geometry)
        box = json.loads(box_str)
        center = centroid(box)
        loc = [center['coordinates'][1], center['coordinates'][0]]
        nom_c = cantons["features"][i]["properties"]["nom"]
        if nom_c in variations_can:
            pop = f'{nom_c}</br>{round(variations_can[nom_c], 4)}'
            if variations_can[nom_c] < 0:
                folium.Marker(location = loc,
                                  popup = pop,
                                  icon = folium.features.CustomIcon('icon/down.png',icon_size=(12,12))).add_to(average_can)
            elif 0 < variations_can[nom_c] <= max(variations_can.values())/1.5:
                folium.Marker(location = loc,
                                  popup = pop,
                                  icon = folium.features.CustomIcon('icon/up_1.png',icon_size=(12,12))).add_to(average_can)
            elif max(variations_can.values())/1.5 < variations_can[nom_c] <= max(variations_can.values()):
                folium.Marker(location = loc,
                                  popup = pop,
                                  icon = folium.features.CustomIcon('icon/up_2.png',icon_size=(12,12))).add_to(average_can)
            else:
                folium.Marker(location = loc,
                                  popup = nom_c,
                                  icon = folium.features.CustomIcon('icon/equal.png',icon_size=(16,16))).add_to(average_can)
                
average_can